In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

import numpy as np
from sklearn.datasets import make_classification
from sklearn import svm

In [ ]:
def evaluateBinaryClassification(predictions, actuals):
    list = []
    contigency = pd.crosstab(actuals,predictions)
    TP = contigency['yes']['yes']
    TN = contigency['no']['no']
    FP = contigency['yes']['no']
    FN = contigency['no']['yes']
    n = contigency.sum().sum()

    Acuracy = (TP + TN)/n
    Recall = TP/(TP+FN)
    Precision = TP/(TP+FP)
    FScore = 2*Recall*Precision/(Recall+Precision)
    list.append(Acuracy)
    list.append(Recall)
    list.append(Precision)
    list.append(FScore)
    # return Acuracy, Recall, Precision, FScore
    return list

In [ ]:
# def evaluateBinaryClassification(predictions, actuals):
#     list = []
#     TP = 0
#     TN = 0 
#     FP = 0 
#     FN = 0 
#     n = np.size(predictions)
#     for i in range(0,n):
#       if predictions[i] == 'yes' and actuals[i] == 'yes':
#         TP = TP + 1
#       elif predictions[i] == 'yes' and actuals[i] == 'no':
#         FP = FP + 1
#       elif predictions[i] == 'no' and actuals[i] == 'no':
#         TN = TN + 1
#       elif predictions[i] == 'no' and actuals[i] == 'yes':
#         FN = FN + 1


#     Acuracy = (TP + TN)/n
#     Recall = TP/(TP+FN)
#     Precision = TP/(TP+FP)
#     FScore = 0
#     if Recall+Precision != 0:
#       Fscore = 2*Recall*Precision/(Recall+Precision)
#     list.append(Acuracy)
#     list.append(Recall)
#     list.append(Precision)
#     list.append(Fscore)
#     # return Acuracy, Recall, Precision, FScore
#     return list

In [ ]:
SEED = 40
scores = []

In [ ]:
data = pd.read_excel(r"/content/2022Binance_Original_LowCloseRemoved_All_Change_num_btc_bin_2day.xlsx", na_values='?',index_col=0)
print(data.dtypes)
data.head()

In [ ]:
def yesno(x):
  if x == 1:
    return 'yes'
  else:
    return 'no'

In [ ]:
data['Binance_change'] = data['Binance_change'].apply(yesno)

In [ ]:
data

In [ ]:
# data = data.drop(['Internet_Penetration_USA','Population_USA','GDP_USA','Inflation_USA','Nasdaq_High','Nasdaq_Low','Nasdaq_Close','Nasdaq_Adj Close','S&P500_High','S&P500_Low','S&P500_Close','S&P500_Adj Close','BTC_High','BTC_Low','BTC_Close','BTC_Market_Cap','BTC_Open'],axis=1)

In [ ]:
# data= data.drop(['BTC_Change_Bin_1day','Internet_Penetration_USA','Population_USA','GDP_USA','Inflation_USA','Gold_Change_bin_1day','Oil_Change_Bin_1day','Nasdaq_Change_bin_1day','S&P500_Change_Bin_1day','BTC_Volume_Change_bin_1day','BTC_Market_Cap_Bin_1day'], axis=1) #date dropped

In [ ]:
data.columns

In [ ]:
Metrics = ['Accuracy','Recall','Precision','Fscore']

compare_df = pd.DataFrame(columns = Metrics)

In [ ]:
s = data.shape

In [ ]:
T = s[0]*0.75 + 1
T = int(T)

KNeighbours Classifier

In [ ]:
# x = np.arange(1,100,1)
# for k in x:
acc = 0
recall = 0
prec = 0
fscore = 0;

for i in range(0,10):
    # Get the train-test subsets
  X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
  y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]

  # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  model = KNeighborsClassifier(p = 2, n_neighbors=17)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  acc += model.score(X_test, y_test)
  recall += recall_score(y_test, y_pred, average='micro')  # average='micro' added to resolve error
  prec += precision_score(y_test, y_pred, average='micro')
  fscore += f1_score(y_test,y_pred, average='micro')
    
acc /= 10
recall /= 10
prec /= 10
fscore /= 10

compare_df.loc['KNeighbors K=13 75-25'] = [acc,recall,prec,fscore]
# print(f"k={k}\tacc={acc}\trecall={recall}\tprec={prec}")
# print(compare_df)
    

Logistic Regression

In [ ]:
from sklearn import datasets, linear_model, metrics
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0

for i in range(0,10):

               # Get the train-test subsets
  X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
  y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # create logistic regression object
  reg = linear_model.LogisticRegression()
    
  # train the model using the training sets
  reg.fit(X_train, y_train)
    
  # making predictions on the testing set 
  y_predict_LR = reg.predict(X_test)
  # print(y_predict_LR)

  list = evaluateBinaryClassification(y_predict_LR,y_test)
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['LogisticRegression'] = evaluateBinaryClassification(y_predict_LR,y_test)
  # print(compare_df)

avg_accuracy /= 10
avg_recall /= 10
avg_precision /= 10
avg_fscore /= 10
compare_df.loc['Logistic Regression 75-25'] = [avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

Support Vector Machine

In [ ]:

avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
for i in range(0,10):

               # Get the train-test subsets
  X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
  y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  model = svm.SVC(kernel = 'linear', random_state = 0, C=3.0)
  model.fit(X_train, y_train)
  y_predict_SVM = model.predict(X_test)
  # print(y_predict_SVM)
  
  list = evaluateBinaryClassification(y_predict_SVM,y_test)
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['SVM'] = evaluateBinaryClassification(y_predict_SVM,y_test)
  # print(compare_df)

avg_accuracy /= 10
avg_recall /= 10
avg_precision /= 10
avg_fscore /= 10
compare_df.loc['SVM 75-25'] = [avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

Gradient Boosting

In [ ]:
avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0
max_accuracy = 0;
for i in range(0,10):

               # Get the train-test subsets
  X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
  y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  model = GradientBoostingClassifier()
  model.fit(X_train, y_train)

  y_predict_GradientBoostingClassifier = model.predict(X_test)
  # print(y_predict_GradientBoostingClassifier)

  list = evaluateBinaryClassification(y_predict_GradientBoostingClassifier,y_test)
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['GradientBoostingClassifier'] = evaluateBinaryClassification(y_predict_GradientBoostingClassifier,y_test)
  # print(compare_df)

avg_accuracy /= 10
avg_recall /= 10
avg_precision /= 10
avg_fscore /= 10
compare_df.loc['Gradient Boosting 75-25'] = [avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0

for i in range(0,10):

               # Get the train-test subsets
  X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
  y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  lda = LinearDiscriminantAnalysis()          #not working for n_components=2
  X_train = lda.fit_transform(X_train, y_train)
  X_test = lda.transform(X_test)

  classifier = RandomForestClassifier(max_depth=3, random_state=0)
  classifier.fit(X_train, y_train)

  #plotDecisionTree(classTree, feature_names=X_train.columns, class_names=classTree.classes_)
  y_predict_LDA = classifier.predict(X_test)
  # print(y_predict_LDA)

  list = evaluateBinaryClassification(y_predict_LDA,y_test)
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['LDA'] = evaluateBinaryClassification(y_predict_LDA,y_test)
  # print(compare_df)

avg_accuracy /= 10
avg_recall /= 10
avg_precision /= 10
avg_fscore /= 10
compare_df.loc['LDA 75-25'] = [avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

LGMB Classifier

In [ ]:

avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0

for i in range(0,10):


               # Get the train-test subsets
  X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
  y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  model = LGBMClassifier()
  model.fit(X_train, y_train)
  y_predict_LGMBclassifier = model.predict(X_test)
  # print(y_predict_LGMBclassifier)

  list = evaluateBinaryClassification(y_predict_LGMBclassifier,y_test)
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  # compare_df.loc['LGBMclassifier'] = evaluateBinaryClassification(y_predict_LGMBclassifier,y_test)
  # print(compare_df)

avg_accuracy /= 10
avg_recall /= 10
avg_precision /= 10
avg_fscore /= 10
compare_df.loc['LGBMclassifier 75-25'] = [avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

XG BOOST

In [ ]:
# avg_accuracy = 0
# avg_recall = 0
# avg_precision = 0
# avg_fscore = 0
# max_accuracy = 0;
# for i in range(0,10):

#                # Get the train-test subsets
#   X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
#   y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]
#         # Scale the X features according to training data 
#   scaler = StandardScaler()
#   X_train = scaler.fit_transform(X_train)
#   X_test = scaler.transform(X_test)

#   model = XGBClassifier()
#   model.fit(X_train, y_train)
#   y_predict_XgBoost = model.predict(X_test)
#   # print(y_predict_XgBoost)

#   list = evaluateBinaryClassification(y_predict_XgBoost,y_test)
#   avg_accuracy += list[0]
#   avg_recall += list[1]
#   avg_precision += list[2]
#   avg_fscore += list[3]

#   # compare_df.loc['XgBoost'] = evaluateBinaryClassification(y_predict_XgBoost,y_test)
#   # print(compare_df)

# avg_accuracy /= 10
# avg_recall /= 10
# avg_precision /= 10
# avg_fscore /= 10
# compare_df.loc['XgBoost 75-25'] = [avg_accuracy,avg_recall,avg_precision,avg_fscore]
# # print(compare_df)

MLP Classifier

In [ ]:

avg_accuracy = 0
avg_recall = 0
avg_precision = 0
avg_fscore = 0

for i in range(0,10):

               # Get the train-test subsets
  X_train, X_test = data.iloc[0:T,:-1], data.iloc[T:,:-1]
  y_train, y_test = data.iloc[0:T,-1], data.iloc[T:,-1]

        # Scale the X features according to training data 
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)


  mlp = MLPClassifier()
  mlp.fit(X_train, y_train)
  y_predict_mlp = mlp.predict(X_test)

  list = evaluateBinaryClassification(y_predict_mlp,y_test)
  avg_accuracy += list[0]
  avg_recall += list[1]
  avg_precision += list[2]
  avg_fscore += list[3]

  #compare_df = pd.DataFrame()
  # compare_df.loc['MLP'] = evaluateBinaryClassification(y_predict_mlp,y_test)
  # print(compare_df)

avg_accuracy /= 10
avg_recall /= 10
avg_precision /= 10
avg_fscore /= 10
compare_df.loc['MLP 75-25'] = [avg_accuracy,avg_recall,avg_precision,avg_fscore]
# print(compare_df)

In [ ]:
print(compare_df)